In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조

/content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조


In [4]:
raw_data = pd.read_csv('common_data1.csv')
raw_data.head(3)

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타혐오,악플/욕설,clean,분쟁유발
0,문재앙은 김정은의 운전사도 안되는 똥휴지다 개돼지들이 뽑은 재앙민국이 망해간다,0,0,0,0,0,0,0,0,1,0,1.0
1,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이제보니 장난아니고 일 재앙...,0,0,0,0,0,0,0,0,0,0,1.0
2,공무뭔들도 정권 바뀐다는걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이지 ...,0,0,0,0,0,0,0,0,1,0,1.0


In [5]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 69.8 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-06-13 07:45:43--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Sign

In [6]:
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
%cd /content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조

mecab = Mecab()
f = open('stopwords.txt', 'r')
stopwords = []
while True:
  temp = f.readline().strip()
  if not temp:
    break
  stopwords.append(temp)

/content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조


In [7]:
# 데이터셋 나누기는 해야함
raw_data = raw_data.sample(len(raw_data))
X_train = raw_data.iloc[:37925, 0]
y_train = raw_data.iloc[:37925, 1:]
X_test = raw_data.iloc[37925:, 0]
y_test = raw_data.iloc[37925:, 1:]

In [8]:
train_data = []
for sentence in tqdm(X_train):
  morphs = mecab.morphs(sentence)
  tmp_X = [word for word in morphs if word not in stopwords]
  train_data.append(tmp_X)

  0%|          | 0/37925 [00:00<?, ?it/s]

In [9]:
test_data = []
for sentence in tqdm(X_test):
  morphs = mecab.morphs(sentence)
  tmp_X = [word for word in morphs if word not in stopwords]
  test_data.append(tmp_X)

  0%|          | 0/9482 [00:00<?, ?it/s]

In [10]:
import pickle

t = pd.read_pickle('tokenizer.pkl')

In [11]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)
X_train = t.texts_to_sequences(train_data)
X_test = t.texts_to_sequences(test_data)

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 60
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

X_train.shape, X_test.shape

((37925, 60), (9482, 60))

In [13]:
vocab_size = len(t.word_index) + 1
vocab_size

40716

In [31]:
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D, Dropout, Concatenate, MaxPooling1D, GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

Model 1

*   Embedding
*   Conv1D
*   Conv1D



In [15]:
model1 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    Conv1D(64, 5, padding='valid', activation='relu', strides=1),
    Dropout(0.5),
    Conv1D(32, 5, padding='valid', activation='relu', strides=1),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dense(11, activation='softmax')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 128)           5211648   
                                                                 
 conv1d (Conv1D)             (None, 56, 64)            41024     
                                                                 
 dropout (Dropout)           (None, 56, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 52, 32)            10272     
                                                                 
 dropout_1 (Dropout)         (None, 52, 32)            0         
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                        

In [16]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1_path = 'best-cnn.h5py'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [17]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, 
    batch_size=128,
    callbacks=[mc1, es1], 
    verbose=0
)


Epoch 1: val_loss improved from inf to 390.75558, saving model to best-cnn.h5py
INFO:tensorflow:Assets written to: best-cnn.h5py/assets

Epoch 2: val_loss did not improve from 390.75558

Epoch 3: val_loss did not improve from 390.75558

Epoch 4: val_loss did not improve from 390.75558

Epoch 5: val_loss did not improve from 390.75558

Epoch 6: val_loss did not improve from 390.75558


In [18]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

297/297 [==============================] - 1s 3ms/step - loss: 393.6883 - accuracy: 0.1145


[393.6882629394531, 0.11453279852867126]

Model 2

*   Embedding
*   BiLSTM



In [19]:
from tensorflow.keras.layers import Bidirectional

In [20]:
model2 = Sequential([ 
    Embedding(vocab_size, 200, input_length=max_len),
    Bidirectional(LSTM(200, return_sequences=True)),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(11, activation='softmax')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 200)           8143200   
                                                                 
 bidirectional (Bidirectiona  (None, 60, 400)          641600    
 l)                                                              
                                                                 
 dropout_2 (Dropout)         (None, 60, 400)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 400)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 11)                4411      
                                                                 
Total params: 8,789,211
Trainable params: 8,789,211
No

In [21]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2_path = 'best-biLSTM.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [22]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=128, callbacks=[mc2,es2]
)

Epoch 1/30
237/238 [============================>.] - ETA: 0s - loss: 2.7274 - accuracy: 0.2068
Epoch 1: val_loss improved from inf to 2.52740, saving model to best-biLSTM.h5
238/238 [==============================] - 19s 44ms/step - loss: 2.7272 - accuracy: 0.2068 - val_loss: 2.5274 - val_accuracy: 0.3535
Epoch 2/30
238/238 [==============================] - ETA: 0s - loss: 2.1485 - accuracy: 0.4319
Epoch 2: val_loss improved from 2.52740 to 2.44573, saving model to best-biLSTM.h5
238/238 [==============================] - 7s 31ms/step - loss: 2.1485 - accuracy: 0.4319 - val_loss: 2.4457 - val_accuracy: 0.4115
Epoch 3/30
238/238 [==============================] - ETA: 0s - loss: 1.9383 - accuracy: 0.5235
Epoch 3: val_loss improved from 2.44573 to 2.27392, saving model to best-biLSTM.h5
238/238 [==============================] - 7s 27ms/step - loss: 1.9383 - accuracy: 0.5235 - val_loss: 2.2739 - val_accuracy: 0.4918
Epoch 4/30
238/238 [==============================] - ETA: 0s - loss: 

In [23]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

297/297 [==============================] - 3s 7ms/step - loss: 2.2605 - accuracy: 0.4880


[2.260547637939453, 0.48797720670700073]

Model 3

*   Embedding
*   LSTM



In [24]:
model3 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    LSTM(128),
    Dense(11, activation='softmax')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_3 (Dense)             (None, 11)                1419      
                                                                 
Total params: 5,344,651
Trainable params: 5,344,651
Non-trainable params: 0
_________________________________________________________________


In [25]:
model3.compile('adam', 'categorical_crossentropy', ['accuracy'])
model3_path = 'best-lstm.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [28]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc3, es3]
)

Epoch 1/100
470/475 [============================>.] - ETA: 0s - loss: 2.9667 - accuracy: 0.1167
Epoch 1: val_loss improved from inf to 2.97062, saving model to best-lstm.h5
475/475 [==============================] - 6s 12ms/step - loss: 2.9682 - accuracy: 0.1164 - val_loss: 2.9706 - val_accuracy: 0.1175
Epoch 2/100
473/475 [============================>.] - ETA: 0s - loss: 2.9684 - accuracy: 0.1164
Epoch 2: val_loss did not improve from 2.97062
475/475 [==============================] - 5s 11ms/step - loss: 2.9685 - accuracy: 0.1164 - val_loss: 2.9743 - val_accuracy: 0.1175
Epoch 3/100
475/475 [==============================] - ETA: 0s - loss: 2.9675 - accuracy: 0.1158
Epoch 3: val_loss did not improve from 2.97062
475/475 [==============================] - 9s 18ms/step - loss: 2.9675 - accuracy: 0.1158 - val_loss: 2.9822 - val_accuracy: 0.1175
Epoch 4/100
473/475 [============================>.] - ETA: 0s - loss: 2.9689 - accuracy: 0.1164
Epoch 4: val_loss improved from 2.97062 to 2.

In [29]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

297/297 [==============================] - 2s 5ms/step - loss: 2.9528 - accuracy: 0.1145


[2.952800750732422, 0.11453279852867126]

Model 4

*   Embedding
*   Conv1D
*   LSTM

In [32]:
model4 = Sequential([
    Embedding(vocab_size, 128, input_length=max_len),
    Dropout(0.2),
    Conv1D(256, 3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=4),
    LSTM(128),
    Dense(11, activation='softmax')
])
model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 dropout_4 (Dropout)         (None, 60, 128)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 58, 256)           98560     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 256)          0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dense_4 (Dense)             (None, 11)                1419      
                                                      

In [33]:
model4.compile('adam', 'categorical_crossentropy', ['accuracy'])
model4_path = 'best-conv1d_lstm.h5'
mc4 = ModelCheckpoint(model4_path, verbose=1, save_best_only=True)
es4 = EarlyStopping(patience=5)

In [34]:
hist4 = model4.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc4, es4]
)

Epoch 1/100
475/475 [==============================] - ETA: 0s - loss: 2.5072 - accuracy: 0.2943
Epoch 1: val_loss improved from inf to 2.06243, saving model to best-conv1d_lstm.h5
475/475 [==============================] - 8s 13ms/step - loss: 2.5072 - accuracy: 0.2943 - val_loss: 2.0624 - val_accuracy: 0.5318
Epoch 2/100
470/475 [============================>.] - ETA: 0s - loss: 1.8320 - accuracy: 0.6030
Epoch 2: val_loss improved from 2.06243 to 1.93665, saving model to best-conv1d_lstm.h5
475/475 [==============================] - 5s 11ms/step - loss: 1.8322 - accuracy: 0.6032 - val_loss: 1.9367 - val_accuracy: 0.5999
Epoch 3/100
475/475 [==============================] - ETA: 0s - loss: 1.7146 - accuracy: 0.6542
Epoch 3: val_loss did not improve from 1.93665
475/475 [==============================] - 5s 11ms/step - loss: 1.7146 - accuracy: 0.6542 - val_loss: 2.0754 - val_accuracy: 0.6082
Epoch 4/100
471/475 [============================>.] - ETA: 0s - loss: 1.7031 - accuracy: 0.68

In [35]:
best_model4 = load_model(model4_path)
best_model4.evaluate(X_test, y_test)

297/297 [==============================] - 2s 4ms/step - loss: 1.9121 - accuracy: 0.6053


[1.912076473236084, 0.6052520275115967]

Model 4

*   Embedding
*   GRU

In [36]:
model5 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    GRU(128),
    Dense(11, activation='softmax')
])
model5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_5 (Dense)             (None, 11)                1419      
                                                                 
Total params: 5,312,139
Trainable params: 5,312,139
Non-trainable params: 0
_________________________________________________________________


In [37]:
model5.compile('adam', 'categorical_crossentropy', ['accuracy'])
model5_path = 'best-gru.h5'
mc5 = ModelCheckpoint(model5_path, verbose=1, save_best_only=True)
es5 = EarlyStopping(patience=5)

In [39]:
hist5 = model5.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc5, es5]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.9829 - accuracy: 0.1165
Epoch 1: val_loss improved from inf to 2.97242, saving model to best-gru.h5
475/475 [==============================] - 11s 18ms/step - loss: 2.9828 - accuracy: 0.1165 - val_loss: 2.9724 - val_accuracy: 0.1175
Epoch 2/100
470/475 [============================>.] - ETA: 0s - loss: 2.9691 - accuracy: 0.1164
Epoch 2: val_loss did not improve from 2.97242
475/475 [==============================] - 5s 11ms/step - loss: 2.9693 - accuracy: 0.1164 - val_loss: 2.9749 - val_accuracy: 0.1175
Epoch 3/100
473/475 [============================>.] - ETA: 0s - loss: 2.9677 - accuracy: 0.1158
Epoch 3: val_loss did not improve from 2.97242
475/475 [==============================] - 6s 12ms/step - loss: 2.9678 - accuracy: 0.1158 - val_loss: 2.9823 - val_accuracy: 0.1175
Epoch 4/100
471/475 [============================>.] - ETA: 0s - loss: 2.9692 - accuracy: 0.1164
Epoch 4: val_loss improved from 2.97242 to 2.

In [40]:
best_model5 = load_model(model5_path)
best_model5.evaluate(X_test, y_test)

297/297 [==============================] - 2s 4ms/step - loss: 2.9528 - accuracy: 0.1145


[2.952824354171753, 0.11453279852867126]

Model 5

*   Embedding
*   Conv1D
*   GRU

In [41]:
model6 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    Dropout(0.2),
    Conv1D(256, 3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=4),
    GRU(128),
    Dense(11, activation='softmax')
])
model6.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 dropout_5 (Dropout)         (None, 60, 128)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 58, 256)           98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 256)          0         
 1D)                                                             
                                                                 
 gru_1 (GRU)                 (None, 128)               148224    
                                                                 
 dense_6 (Dense)             (None, 11)                1419      
                                                      

In [42]:
model6.compile('adam', 'categorical_crossentropy', ['accuracy'])
model6_path = 'best-conv1d_gru.h5'
mc6 = ModelCheckpoint(model6_path, verbose=1, save_best_only=True)
es6 = EarlyStopping(patience=5)

In [43]:
hist6 = model6.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc6, es6]
)

Epoch 1/100
471/475 [============================>.] - ETA: 0s - loss: 2.3482 - accuracy: 0.3759
Epoch 1: val_loss improved from inf to 1.86814, saving model to best-conv1d_gru.h5
475/475 [==============================] - 7s 11ms/step - loss: 2.3461 - accuracy: 0.3773 - val_loss: 1.8681 - val_accuracy: 0.6030
Epoch 2/100
472/475 [============================>.] - ETA: 0s - loss: 1.7007 - accuracy: 0.6677
Epoch 2: val_loss did not improve from 1.86814
475/475 [==============================] - 5s 10ms/step - loss: 1.7012 - accuracy: 0.6678 - val_loss: 1.8712 - val_accuracy: 0.6411
Epoch 3/100
471/475 [============================>.] - ETA: 0s - loss: 1.6382 - accuracy: 0.7172
Epoch 3: val_loss did not improve from 1.86814
475/475 [==============================] - 4s 9ms/step - loss: 1.6381 - accuracy: 0.7170 - val_loss: 2.1357 - val_accuracy: 0.6450
Epoch 4/100
473/475 [============================>.] - ETA: 0s - loss: 1.6493 - accuracy: 0.7417
Epoch 4: val_loss did not improve from 1

In [44]:
best_model6 = load_model(model6_path)
best_model6.evaluate(X_test, y_test)

297/297 [==============================] - 1s 4ms/step - loss: 1.8488 - accuracy: 0.6069


[1.848807454109192, 0.606939435005188]